Collaboration and ideas: Gabriele Pirilli and some coding help from Chat GPT

In [ ]:
import heapq
import numpy as np
from collections import namedtuple
from icecream import ic
from tqdm import tqdm
from random import choice
import random

***REMARK:*** For n >= 4 it becomes very slow. For n = 4 it converges after a lot of time and it is faster for some seads, but slower for others.

## INITIALIZATIONS

In [14]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])   #From position 1 to position 2             => Class-like structure

Functions for the actions:

In [3]:
def available_actions(state: tuple) -> list[action]:
    x, y = [(i, row.index(0)) for i, row in enumerate(state) if 0 in row][0]
    actions = []
    if x > 0:  
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:  
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:  
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:  
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: tuple, act: action) -> tuple:
    state_list = [list(row) for row in state]  
    state_list[act.pos1[0]][act.pos1[1]], state_list[act.pos2[0]][act.pos2[1]] = \
        state_list[act.pos2[0]][act.pos2[1]], state_list[act.pos1[0]][act.pos1[1]]
    return tuple(tuple(row) for row in state_list) 

Problem Initialization:

In [4]:
seed = 11
np.random.seed(seed)  
random.seed(seed) 


# The goal state is a tuple of tuples
goal_state_array = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal_state = tuple(tuple(row) for row in goal_state_array)    #Tuples will be faster later

# Generate the initial state by randomizing the goal state
RANDOMIZE_STEPS = 100_000
initial_state = goal_state
for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))

Randomizing: 100%|█████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 323180.72it/s]


In [5]:
print('goal_state \n', goal_state)
print('initial_state \n',initial_state)

goal_state 
 ((np.int64(1), np.int64(2), np.int64(3), np.int64(4)), (np.int64(5), np.int64(6), np.int64(7), np.int64(8)), (np.int64(9), np.int64(10), np.int64(11), np.int64(12)), (np.int64(13), np.int64(14), np.int64(15), np.int64(0)))
initial_state 
 ((np.int64(2), np.int64(9), np.int64(1), np.int64(5)), (np.int64(10), np.int64(13), np.int64(15), np.int64(11)), (np.int64(14), np.int64(6), np.int64(0), np.int64(12)), (np.int64(7), np.int64(8), np.int64(3), np.int64(4)))


## A* IMPLEMENTATION

***f = g + h*** <br>
- g: cost of the path from the start to the current position
- h: heuristic estimate of the cost to reach the final position => in this case, ***Manhattan distance***

<br>
Priority queue => The nodes are explored starting from those with the lowest cost to those with higher costs, 0 is the highest priority
<br>

***goal_test***  => Is the state the final one?

In [6]:
def manhattan_dist(state: tuple, goal: tuple) -> int:
    dist = 0
    for num in range(1, PUZZLE_DIM**2):  # 0 is not considered
        x1, y1 = [(i, row.index(num)) for i, row in enumerate(state) if num in row][0]
        x2, y2 = [(i, row.index(num)) for i, row in enumerate(goal) if num in row][0]
        dist += abs(x1 - x2) + abs(y1 - y2)
    return dist

def goal_test(current_state: tuple, goal_state: tuple) -> bool:
    return current_state == goal_state

***stored_manhattan_dist*** contains already calculated manhattan distances to avoid recalculating them.

In [7]:
manhattan_storing = {}  # Storing Manhattan distances

def stored_manhattan_dist(state: np.ndarray, goal: np.ndarray) -> int:
    state_str = str(state)

    if state_str not in manhattan_storing:    # If not present, compute and store
        manhattan_storing[state_str] = manhattan_dist(state, goal)

    return manhattan_storing[state_str]

In [8]:
def A_star(initial_state: tuple, goal_state: tuple):
    Frontier_states = []  # Priority Queue
    heapq.heappush(Frontier_states, (0, initial_state))

    History = {str(initial_state): None}  # Dictionary for parent tracking
    g_cost = {str(initial_state): 0}
    Examined_states = set()

    n_it_max = 10_000_000
    n_it = 0
    while Frontier_states and n_it < n_it_max:
        n_it += 1
        if n_it % 10_000 == 0:
            print(f"Iteration: {n_it}")

        _, current_state = heapq.heappop(Frontier_states)

        if goal_test(current_state, goal_state):  # If goal state is reached
            print("Goal reached!")
            print('Iterations:', n_it)

            # Reconstruct the path from start to goal
            path = []
            state = current_state
            while state is not None:
                path.append(state)
                state = History[str(state)]
            path.reverse()
            return path, n_it

        Examined_states.add(str(current_state))

        for act in available_actions(current_state):
            neighbor = do_action(current_state, act)
            if str(neighbor) in Examined_states:
                continue

            new_g_cost = g_cost[str(current_state)] + 1
            if str(neighbor) not in g_cost or new_g_cost < g_cost[str(neighbor)]:
                g_cost[str(neighbor)] = new_g_cost
                h_cost = manhattan_dist(neighbor, goal_state)
                f_cost = new_g_cost + h_cost
                heapq.heappush(Frontier_states, (f_cost, neighbor))
                History[str(neighbor)] = current_state

    print("No solution found.")
    return None

REMARK: <br>
*_, current_state_tuple = heapq.heappop(Frontier_states)  <br>
current_state = np.array(current_state_tuple)* <br>

*heapq.heappush(Frontier_states, (f_cost, tuple(map(tuple, neighbor))))* <br>

 => ***tuples*** are needed to make the comparison between priorities of the frontier nodes (with numpy arrays it doesn't work)

In [9]:
seed = 11
np.random.seed(seed) 
random.seed(seed) 

path, n_it = A_star(initial_state, goal_state)

Iteration: 10000
Iteration: 20000
Iteration: 30000
Iteration: 40000
Iteration: 50000
Iteration: 60000
Iteration: 70000
Iteration: 80000
Iteration: 90000
Iteration: 100000
Iteration: 110000
Iteration: 120000
Iteration: 130000
Iteration: 140000
Iteration: 150000
Iteration: 160000
Iteration: 170000
Iteration: 180000
Iteration: 190000
Iteration: 200000
Iteration: 210000
Iteration: 220000
Iteration: 230000
Iteration: 240000
Iteration: 250000
Iteration: 260000
Iteration: 270000
Iteration: 280000
Iteration: 290000
Iteration: 300000
Iteration: 310000
Iteration: 320000
Iteration: 330000
Iteration: 340000
Iteration: 350000
Iteration: 360000
Iteration: 370000
Iteration: 380000
Iteration: 390000
Iteration: 400000
Iteration: 410000
Iteration: 420000
Iteration: 430000
Iteration: 440000
Iteration: 450000
Iteration: 460000
Iteration: 470000
Iteration: 480000
Iteration: 490000
Iteration: 500000
Iteration: 510000
Iteration: 520000
Iteration: 530000
Iteration: 540000
Iteration: 550000
Iteration: 560000
I

In [10]:
print('Quality: ', len(path))
print('Cost', n_it)

Quality:  53
Cost 4326999
